In [9]:
import tkinter as tk
from tkinter import *
from tkinter import messagebox
from tkinter import ttk
import time
import requests
import re
import math
import sqlite3
from multiprocessing import Process
# НАШИ ПАРСЕРЫ
import parser_gulliver
import parser_crockid
import parser_gloria
import parser_acoola
#
from multiprocessing import Pool
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import os
import urllib.request
import re
import pandas as pd
import datetime
headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.81 Safari/537.36 Edg/104.0.1293.54"}

In [2]:
def Interface():
    def callbackFunc(event):
        global urlbinance
        url1 = comboExample.get()
        if(url1 == "Gulliver"):  
            gulliver()
            messagebox.showinfo("Выполнено!")
        if(url1 == "Gloria"):  
            gloria()
            messagebox.showinfo("Выполнено!")
        if(url1 == "Sela"):  
            sela()
            messagebox.showinfo("Выполнено!")
        if(url1 == "Crockid"):  
            crockid()
            messagebox.showinfo("Выполнено!")
        if(url1 == "Acoola"):  
            acoola()
            messagebox.showinfo("Выполнено!")
    pars = tk.Tk()
    pars.geometry('300x300')
    pars.title('Программа для парсинга')
    title = Label(pars, text='Программа для арбитража', font='Arial 40')

    labelTop = tk.Label(pars,
                text = "Какой сайт хотите спарсить?")
    labelTop.pack()
    comboExample = ttk.Combobox(pars, 
                        values=[
                                "Gulliver", 
                                "Gloria",
                                "Sela",
                                "Crockid",
                                "Acoola"],state="readonly")
    comboExample.pack()
    comboExample.bind("<<ComboboxSelected>>", callbackFunc)
    pars.mainloop()


In [3]:
def gulliver():
    link = "https://www.gulliver.ru/catalog/odezhda/b/gulliver?sort=our_choice,asc"
    page = requests.get(link)
    soup = BeautifulSoup(page.text, 'html.parser')
    dec2 = soup.find_all('a',{'class':"page-pagination__link"})[-1:]
    dictionary = {}
    for item in dec2:
          amount_pages = item.text
    pool = Pool()
    pool.map(parser_gulliver.parser, range(int(amount_pages)))    

In [4]:
def gloria():
#ДЕТИ
    link = "https://www.gloria-jeans.ru/c/kids"
    page = requests.get(link)
    soup = BeautifulSoup(page.text, 'html.parser')
    dec2 = soup.find_all('a',{'class':"uk-pagination__number js-pagination-link"})[-1:]
    for item in dec2:
          amount_pages=item.text
    pool = Pool()
    pool.map(parser_gloria.parser, range(int(amount_pages)))
#ПОДРОСТКИ
    link = "https://www.gloria-jeans.ru/c/teenagers"
    page = requests.get(link)
    soup = BeautifulSoup(page.text, 'html.parser')
    dec2 = soup.find_all('a',{'class':"uk-pagination__number js-pagination-link"})[-1:]
    for item in dec2:
          amount_pages=item.text
    pool = Pool()
    pool.map(parser_gloria.parser2, range(int(amount_pages)))

In [5]:
def crockid():
#МАЛЬЧИКИ
    link = "https://www.crockid.ru/catalog/odezhda-dlya-malchikov"
    page = requests.get(link)
    soup = BeautifulSoup(page.text, 'html.parser')
    dec = soup.find_all('div',{'class':"paginateBlock"})
    dec_mas = []
    for item in dec:
        dec_mas.append(item.text.strip())
    amount_pages=re.findall(r'\.(\d+)\s',dec_mas[0])
    pool = Pool()
    pool.map(parser_crockid.parser, range(int(amount_pages[0]))) 
#ДЕВОЧКИ
    link = "https://www.crockid.ru/catalog/girls"
    amount_pages=[]
    page = requests.get(link)
    soup = BeautifulSoup(page.text, 'html.parser')
    dec = soup.find_all('div',{'class':"paginateBlock"})
    dec_mas = []
    for item in dec:
        dec_mas.append(item.text.strip())
    amount_pages=re.findall(r'\.(\d+)\s',dec_mas[0])
    pool = Pool()
    pool.map(parser_crockid.parser2, range(int(amount_pages[0])))
#НОВОРОЖДЕННЫЕ
    link = "https://www.crockid.ru/catalog/baby"
    amount_pages=[]
    page = requests.get(link)
    soup = BeautifulSoup(page.text, 'html.parser')
    dec = soup.find_all('div',{'class':"paginateBlock"})
    dec_mas = []
    for item in dec:
        dec_mas.append(item.text.strip())
    amount_pages=re.findall(r'\.(\d+)\s',dec_mas[0])
    pool = Pool()
    pool.map(parser_crockid.parser3, range(int(amount_pages[0]))) 

In [14]:
def parser_sela(conn, driver, n): 
    print(n)
    now = datetime.datetime.now()
    driver.get("https://www.sela.ru/eshop/kids/girl/?page=" + str(n+1) + "&price_min=199&price_max=6999&sort=publish_datetime&order=desc&_=1660812788266")
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    links = soup.find_all('li',{'class':"product-catalog__item js-product-catalog-item"})
    links_mas = []
#ПОЛУЧАЕМ ССЫЛКИ
    for item in links:
        links_mas.append(item.find('a')['href'])
#ПОЛУЧАЕМ КАРТИНКИ
    images_mas = []
    for item in soup.find_all('li',{'class':"product-card__slider-item js-product-card-slide is-active"}):
        if item.find('img').get('src')!=None:
            images_mas.append(item.find('img').get('src'))
    k=0
#ИДЕМ ПО ЦИКЛУ
    for item in links_mas:
        dictionary = {}
        link = "https://www.sela.ru" + item
        dictionary['Ссылка'] = link
        dictionary['Ссылка_картинка'] = images_mas[k]
        dictionary['Дата']=now.strftime("%d-%m-%Y")
        k+=1
        try:
            driver.get(link)
        except TimeoutException:
            driver.get(link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
#ЦЕНЫ
        prices = soup.find_all('div',{'class':"product-page__price"})
        price_mas=""
        price_mas_sale=[]
        for item in prices:
            price_mas+=item.text.strip().replace("\n","|")
        if "|" in price_mas:
            price_mas_sale=re.split(r'\|',price_mas)
            dictionary['Цена'] = price_mas_sale[1]
            dictionary['Цена_со_скидкой'] = price_mas_sale[0]
        else:
            dictionary['Цена']=price_mas
            dictionary['Цена_со_скидкой']="-"
#НАЗВАНИЕ
        name = soup.find_all('h1',{'class':"product-page__title"})
        for item in name:
            dictionary['Наименование'] = item.text.strip()
#БЛОК ДЕТАЛЕЙ
        details_soup = soup.find_all('div',{'style':"margin-top:15px"})
        details = []
        for item in details_soup:
            details.append(item.text.strip())
        if len(details)>=3:
            if "Возраст:" in details[0]:
                dictionary['Возраст']=details[0][details[0].index(":")+2:]
            if "Сезон:" in details[1]:
                dictionary['Сезон']=details[1][details[1].index(":")+2:]
            if "Состав:" in details[2]:
                dictionary['Состав']=details[2][details[2].index(":")+2:]
        else:
            if "Сезон:" in details[0]:
                dictionary['Сезон']=details[0][details[0].index(":")+2:]
            if "Состав:" in details[1]:
                dictionary['Состав']=details[1][details[1].index(":")+2:]
#АРТИКУЛ
        art = soup.find_all('b',{'style':"white-space:nowrap;"})
        for item in art:
             dictionary['Артикул']=item.text.strip()
#БЛОК РАЗМЕРОВ
        size = []
        size_s = ""
        size_ros = ""
        available = []
        av = soup.find_all('span', {'data-omni':"false"})
        for item in av:
            if item.find('span', {'class':"stock-high"})!=None:
                available.append(item.find('span', {'class':"stock-high"}).text.strip().replace("\xa0"," "))
                size.append(re.split(r", ",item['data-sku_name'])[1])
            if item.find('span', {'class':"stock-low"})!=None:
                available.append(item.find('span', {'class':"stock-low"}).text.strip().replace("\xa0"," "))
                size.append(re.split(r", ",item['data-sku_name'])[1])
            if item.find('span', {'class':"stock-critical"})!=None:
                available.append(item.find('span', {'class':"stock-critical"}).text.strip().replace("\xa0"," "))
                size.append(re.split(r", ",item['data-sku_name'])[1])
        av = soup.find_all('span', {'data-omni':"true"})
        for item in av:
            if item.find('span', {'class':"stock-high"})!=None:
                available.append(item.find('span', {'class':"stock-high"}).text.strip().replace("\xa0"," "))
                size.append(re.split(r", ",item['data-sku_name'])[1])
            if item.find('span', {'class':"stock-low"})!=None:
                available.append(item.find('span', {'class':"stock-low"}).text.strip().replace("\xa0"," "))
                size.append(re.split(r", ",item['data-sku_name'])[1])
            if item.find('span', {'class':"stock-critical"})!=None:
                available.append(item.find('span', {'class':"stock-critical"}).text.strip().replace("\xa0"," "))
                size.append(re.split(r", ",item['data-sku_name'])[1])
        size_active = soup.find_all('li', {'class':"product-page__size-item js-size-item is-active-size"})
        for item in size_active:
            size_ac = item.text.strip()
        if size_ac not in size:
            available.append("сообщить о поступлении")
            size.append(size_ac)
        for i in range(len(available)):
            if available[i]=='сообщить о поступлении' or available[i]=='в наличии только в магазинах':
                size_ros+=size[i] + " "
            else:
                size_s+=size[i] + " "
        dictionary['Размер']=size_s
        dictionary['Размер_розница']=size_ros
#ПОЛ/ТИП
        size_active = soup.find_all('span',{'itemprop':"name"})
        if len(size_active)>=3:
            for item in size_active[0]:
                dictionary['Товарная_группа1'] = item.text.strip()
            for item in size_active[1]:
                dictionary['Товарная_группа2'] = item.text.strip()
            for item in size_active[2]:
                dictionary['Товарная_группа3'] = item.text.strip()
        elif len(size_active)==2:
            for item in size_active[0]:
                dictionary['Товарная_группа1'] = item.text.strip()
            for item in size_active[1]:
                dictionary['Товарная_группа2'] = item.text.strip()
        else:
            for item in size_active[0]:
                dictionary['Товарная_группа1'] = item.text.strip()
#ЦВЕТ
        colour = soup.find_all('li',{'class':"product-page__color-item is-border is-active-color"})
        if colour == []:
            colour = soup.find_all('li',{'class':"product-page__color-item is-active-color"})
        for item in colour:
            dictionary['Цвет']=item['title']
#ОПИСАНИЕ
        description = soup.find_all('div', {'class':"product-page__item-text js-content"})[0]
        desc_mas = []
        for item in description:
             desc_mas.append(item.text.strip())
        dictionary['Описание']=desc_mas[2]
#ЗАБИВАЕМ В SQL
        if dictionary['Ссылка'] in ultradata[0] and (dictionary['Цена']!=ultradata[1][ultradata[0].index(dictionary['Ссылка'])] or dictionary['Цена_со_скидкой']!=ultradata[2][ultradata[0].index(dictionary['Ссылка'])]):
            df = pd.DataFrame([dictionary], columns= ['Наименование','Цена','Цена_со_скидкой','Категория','Возраст','Состав','Цвет','Сезон','Артикул','Описание','Товарная_группа1','Товарная_группа2','Товарная_группа3','Размер','Размер_розница','Ссылка','Ссылка_картинка', 'Дата'])
            df.to_sql('sela', conn, if_exists='append', index = False)
        elif dictionary['Ссылка'] not in ultradata[0]:
            df = pd.DataFrame([dictionary], columns= ['Наименование','Цена','Цена_со_скидкой','Категория','Возраст','Состав','Цвет','Сезон','Артикул','Описание','Товарная_группа1','Товарная_группа2','Товарная_группа3','Размер','Размер_розница','Ссылка','Ссылка_картинка', 'Дата'])
            df.to_sql('sela', conn, if_exists='append', index = False)
def parser_sela2(conn, driver, n):
    print(n)
    now = datetime.datetime.now()
    driver.get("https://www.sela.ru/eshop/kids/boy/?page=" + str(n+1) + "&price_min=49&price_max=6999&age%5B0%5D=5068&age%5B1%5D=5067&sort=publish_datetime&order=desc")
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    links = soup.find_all('li',{'class':"product-card__slider-item js-product-card-slide is-active"})
    links_mas = []
#ПОЛУЧАЕМ ССЫЛКИ
    for item in links:
        links_mas.append(item.find('a')['href'])
#ПОЛУЧАЕМ КАРТИНКИ
    images = soup.find_all('li',{'class':"product-card__slider-item js-product-card-slide"})
    images_mas = []
    for item in soup.find_all('li',{'class':"product-card__slider-item js-product-card-slide"}):
        if item.find('img').get('src')!=None:
            images_mas.append(item.find('img').get('src'))
    k=0
    for item in links_mas:
        dictionary = {}
        link = "https://www.sela.ru" + item
        dictionary['Ссылка'] = link
        dictionary['Ссылка_картинка'] = images_mas[k]
        dictionary['Дата']=now.strftime("%d-%m-%Y")
        k+=1
        try:
            driver.get(link)
        except TimeoutException:
            driver.get(link)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
#ЦЕНЫ
        prices = soup.find_all('div',{'class':"product-page__price"})
        price_mas=""
        price_mas_sale=[]
        for item in prices:
            price_mas+=item.text.strip().replace("\n","|")
        if "|" in price_mas:
            price_mas_sale=re.split(r'\|',price_mas)
            dictionary['Цена'] = price_mas_sale[1]
            dictionary['Цена_со_скидкой'] = price_mas_sale[0]
        else:
            dictionary['Цена']=price_mas
            dictionary['Цена_со_скидкой']="-"
#НАЗВАНИЕ
        name = soup.find_all('h1',{'class':"product-page__title"})
        for item in name:
            dictionary['Наименование'] = item.text.strip()
#БЛОК ДЕТАЛЕЙ
        details_soup = soup.find_all('div',{'style':"margin-top:15px"})
        details = []
        for item in details_soup:
            details.append(item.text.strip())
        if len(details)>=3:
            if "Возраст:" in details[0]:
                dictionary['Возраст']=details[0][details[0].index(":")+2:]
            if "Сезон:" in details[1]:
                dictionary['Сезон']=details[1][details[1].index(":")+2:]
            if "Состав:" in details[2]:
                dictionary['Состав']=details[2][details[2].index(":")+2:]
        else:
            if "Сезон:" in details[0]:
                dictionary['Сезон']=details[0][details[0].index(":")+2:]
            if "Состав:" in details[1]:
                dictionary['Состав']=details[1][details[1].index(":")+2:]
#АРТИКУЛ
        art = soup.find_all('b',{'style':"white-space:nowrap;"})
        for item in art:
             dictionary['Артикул']=item.text.strip()
#РАЗМЕРЫ
        size = []
        size_s = ""
        size_ros = ""
        available = []
        av = soup.find_all('span', {'data-omni':"false"})
        for item in av:
            if item.find('span', {'class':"stock-high"})!=None:
                available.append(item.find('span', {'class':"stock-high"}).text.strip().replace("\xa0"," "))
                size.append(re.split(r", ",item['data-sku_name'])[1])
            if item.find('span', {'class':"stock-low"})!=None:
                available.append(item.find('span', {'class':"stock-low"}).text.strip().replace("\xa0"," "))
                size.append(re.split(r", ",item['data-sku_name'])[1])
            if item.find('span', {'class':"stock-critical"})!=None:
                available.append(item.find('span', {'class':"stock-critical"}).text.strip().replace("\xa0"," "))
                size.append(re.split(r", ",item['data-sku_name'])[1])
        av = soup.find_all('span', {'data-omni':"true"})
        for item in av:
            if item.find('span', {'class':"stock-high"})!=None:
                available.append(item.find('span', {'class':"stock-high"}).text.strip().replace("\xa0"," "))
                size.append(re.split(r", ",item['data-sku_name'])[1])
            if item.find('span', {'class':"stock-low"})!=None:
                available.append(item.find('span', {'class':"stock-low"}).text.strip().replace("\xa0"," "))
                size.append(re.split(r", ",item['data-sku_name'])[1])
            if item.find('span', {'class':"stock-critical"})!=None:
                available.append(item.find('span', {'class':"stock-critical"}).text.strip().replace("\xa0"," "))
                size.append(re.split(r", ",item['data-sku_name'])[1])
        size_active = soup.find_all('li', {'class':"product-page__size-item js-size-item is-active-size"})
        for item in size_active:
            size_ac = item.text.strip()
        if size_ac not in size:
            available.append("сообщить о поступлении")
            size.append(size_ac)
        for i in range(len(available)):
            if available[i]=='сообщить о поступлении' or available[i]=='в наличии только в магазинах':
                size_ros+=size[i] + " "
            else:
                size_s+=size[i] + " "
        dictionary['Размер']=size_s
        dictionary['Размер_розница']=size_ros
#ПОЛ/ТИП
        size_active = soup.find_all('span',{'itemprop':"name"})
        if len(size_active)>=3:
            for item in size_active[0]:
                dictionary['Товарная_группа1'] = item.text.strip()
            for item in size_active[1]:
                dictionary['Товарная_группа2'] = item.text.strip()
            for item in size_active[2]:
                dictionary['Товарная_группа3'] = item.text.strip()
        elif len(size_active)==2:
            for item in size_active[0]:
                dictionary['Товарная_группа1'] = item.text.strip()
            for item in size_active[1]:
                dictionary['Товарная_группа2'] = item.text.strip()
        else:
            for item in size_active[0]:
                dictionary['Товарная_группа1'] = item.text.strip()
#ЦВЕТ
        colour = soup.find_all('li',{'class':"product-page__color-item is-border is-active-color"})
        if colour == []:
            colour = soup.find_all('li',{'class':"product-page__color-item is-active-color"})
        for item in colour:
            dictionary['Цвет']=item['title']
#ОПИСАНИЕ
        description = soup.find_all('div', {'class':"product-page__item-text js-content"})[0]
        desc_mas = []
        for item in description:
             desc_mas.append(item.text.strip())
        dictionary['Описание']=desc_mas[2]
#ЗАБИВАЕМ В SQL
        if dictionary['Ссылка'] in ultradata[0] and (dictionary['Цена']!=ultradata[1][ultradata[0].index(dictionary['Ссылка'])] or dictionary['Цена_со_скидкой']!=ultradata[2][ultradata[0].index(dictionary['Ссылка'])]):
            df = pd.DataFrame([dictionary], columns= ['Наименование','Цена','Цена_со_скидкой','Категория','Возраст','Состав','Цвет','Сезон','Артикул','Описание','Товарная_группа1','Товарная_группа2','Товарная_группа3','Размер','Размер_розница','Ссылка','Ссылка_картинка', 'Дата'])
            df.to_sql('sela', conn, if_exists='append', index = False)
        elif dictionary['Ссылка'] not in ultradata[0]:
            df = pd.DataFrame([dictionary], columns= ['Наименование','Цена','Цена_со_скидкой','Категория','Возраст','Состав','Цвет','Сезон','Артикул','Описание','Товарная_группа1','Товарная_группа2','Товарная_группа3','Размер','Размер_розница','Ссылка','Ссылка_картинка', 'Дата'])
            df.to_sql('sela', conn, if_exists='append', index = False)
def sela():
    global ultradata
    conn = sqlite3.connect('database3')
    c = conn.cursor()
    c.execute('CREATE TABLE IF NOT EXISTS sela (Наименование text, Цена number, Цена_со_скидкой number, Возраст text, Состав text, Категория text, Цвет text, Сезон text, Артикул text, Описание text, Товарная_группа1 text, Товарная_группа2 text, Товарная_группа3 text, Размер text, Размер_розница text, Ссылка text,Ссылка_картинка text, Дата text)')
    conn.commit()
    cursor = conn.execute("SELECT Ссылка, Цена, Цена_со_скидкой from sela")
    ultradata = [[] for i in range(3)]
    for row in cursor:
        ultradata[0].append(row[0])
        ultradata[1].append(row[1])
        ultradata[2].append(row[2])
    conn.commit()
#ЗАПУСКАЕМ ДРАЙВЕР
    options = Options()
    options.add_argument('--disable-site-isolation-trials')
    options.w3c = True
    driver = webdriver.Chrome(r'chromedriver.exe', options=options)
#ДЕВОЧКИ
    link = "https://www.sela.ru/eshop/kids/girl/?price_min=199&price_max=6999&age%5B%5D=5068&age%5B%5D=5067&sort=publish_datetime&order=desc"
    driver.get(link)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
#КОЛИЧЕСТВО СТРАНИЦ
    dec2 = soup.find_all('a',{'class':"pagination-wrapper__pagination-list-item"})[-1:]
    for item in dec2:
          amount_pages=item.text
    for n in range(int(amount_pages)):
        parser_sela(conn, driver, n)
#МАЛЬЧИКИ
    link = "https://www.sela.ru/eshop/kids/boy/?price_min=49&price_max=6999&age%5B%5D=5068&age%5B%5D=5067&sort=publish_datetime&order=desc"
    driver.get(link)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
#КОЛИЧЕСТВО СТРАНИЦ
    dec2 = soup.find_all('a',{'class':"pagination-wrapper__pagination-list-item"})[-1:]
    for item in dec2:
          amount_pages=item.text
    for n in range(int(amount_pages)):
        parser_sela2(conn, driver,n)

In [15]:
def acoola():
#ДЕВОЧКИ 2-8
    link = "https://acoolakids.ru/vsya-odezhda-dlya-devochek-2-8"
    page = requests.get(link)
    soup = BeautifulSoup(page.text, 'html.parser')
    dec = soup.find_all('nav',{'class':"ak-pagination"})[-1]
    for item in dec:
        amount_pages=item.text.split('.')[-1]
    pool = Pool()
    pool.map(parser_acoola.parser, range(int(amount_pages)))    
#ДЕВОЧКИ 8-14
    link = "https://acoolakids.ru/vsya-odezhda-dlya-devochek-8-14"
    page = requests.get(link)
    soup = BeautifulSoup(page.text, 'html.parser')
    dec = soup.find_all('nav',{'class':"ak-pagination"})[-1]
    for item in dec:
        amount_pages=item.text.split('.')[-1]
    pool = Pool()
    pool.map(parser_acoola.parser2, range(int(amount_pages)))    
#МАЛЬЧИКИ 2-8
    link = "https://acoolakids.ru/vsya-odezhda-dlya-malchikov-3-8"
    page = requests.get(link)
    soup = BeautifulSoup(page.text, 'html.parser')
    dec = soup.find_all('nav',{'class':"ak-pagination"})[-1]
    for item in dec:
        amount_pages=item.text.split('.')[-1]
    pool = Pool()
    pool.map(parser_acoola.parser3, range(int(amount_pages)))    
#ДЕВОЧКИ 8-14
    link = "https://acoolakids.ru/vsya-odezhda-dlya-malchikov-8-14"
    page = requests.get(link)
    soup = BeautifulSoup(page.text, 'html.parser')
    dec = soup.find_all('nav',{'class':"ak-pagination"})[-1]
    for item in dec:
        amount_pages=item.text.split('.')[-1]
    pool = Pool()
    pool.map(parser_acoola.parser4, range(int(amount_pages)))    

In [18]:
if __name__ == '__main__':
    Interface()

In [ ]:
print(link)